Tutorial on "Anomaly detection". Part Python.

Author: Pavlo Mozharovskyi.

This is a Python Notebook for the tutorial 
on "Anomaly detection" given on Wednesday the 13th of April 2022.

The data set is provided by the Airbus and consistst of 1677 the measures of the accelerometer of helicopters during  1 minute at frequency 1024 Hertz, which yields time series measured at in total 60 * 1024 = 61440 equidistant time points.

## 0) Import packages

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
from FIF import *

## 1) Load and investigate the data

In [ ]:
xtrain = np.loadtxt('../data/airbus_data.csv', delimiter = ' ')
print(xtrain.shape)

### A sample plot

In [ ]:
# Plot first 614 time points for first 100 observations
plt.figure(figsize=(16,8))
for i in range(100):
    plt.plot(range(614),xtrain[i,:614])
plt.show()

## 2) Preprocess data: reduce measurement frequency (attention - possible information loss)

In [ ]:
nArgs = 101
xtrainRed = np.empty([xtrain.shape[0], nArgs])
args = np.linspace(0, xtrain.shape[1] - 1, num = nArgs)
for i in range(0, xtrain.shape[0]):
    xtrainRed[i,:] = np.interp(args, np.array(range(xtrain.shape[1])), xtrain[i,:])
print(xtrainRed.shape)

## 3) Using the projection on a low-dimensional space

In [ ]:
# PCA transform
pca1 = PCA(n_components = 10, whiten = True)
pca1.fit(xtrain)
xtrain_fpca = pca1.fit_transform(xtrainRed)
print(xtrain_fpca.shape)
# Fit the low-dimensional method
lof1 = LocalOutlierFactor(n_neighbors = 5 ,contamination = 'auto', novelty = True)
lof1.fit(xtrain_fpca)
# Calculate anomaly score on the (PCA-transformed) data
lof1_score = -lof1.score_samples(xtrain_fpca)
print(lof1_score)

Plot the outcome

In [ ]:
plt.figure(1, figsize=(15, 5))
# Plot the 2-dimensional projection
plt.subplot(121)
plt.scatter(xtrain_fpca[:,0], xtrain_fpca[:,1], c = lof1_score)
plt.xlabel('1st PC')
plt.ylabel('2nd PC')
plt.subplot(122)
plt.scatter(range(0, lof1_score.shape[0]), np.sort(lof1_score), c = np.sort(lof1_score))
plt.ylabel('LOF novelty score')
plt.xlabel('(Score-ordered) observation number')

## 4) The functional isolation forest

In [ ]:
dicsFIF = [xtrainRed, 'Dyadic_indicator', 'cosinus']
dicTitles = ['Self dictionary', 'Dyadic indicator dictionary', 'Cosine dictionary']
psi = 32
plt.figure(1, figsize=(15, 5))
for i in range(0, len(dicsFIF)):
    F = FIForest(xtrainRed, ntrees = 100, time = np.array(range(nArgs)), subsample_size = psi, 
                 D = dicsFIF[i], innerproduct = "auto", Dsize = 100, alpha = 1)
    fif1_score = F.compute_paths(X_in = xtrainRed)
    plt.subplot(131 + i)
    plt.scatter(range(0, fif1_score.shape[0]), np.sort(fif1_score), c = np.sort(fif1_score))
    plt.ylabel('FIF novelty score')
    plt.xlabel('Observation number')
    plt.title(dicTitles[i])

## ?) Try other mentioned methods for these data